In [18]:
import torch
import torch.nn.functional as F
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GCNConv

In [2]:
from torch_geometric.datasets import Planetoid

In [3]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [4]:
len(dataset)

1

In [5]:
dataset.num_classes

7

In [6]:
dataset.num_node_features

1433

In [7]:
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [13]:
data.is_undirected()

True

In [15]:
# against which nodes to train
data.train_mask.sum().item()

140

In [16]:
# nodes to use for validation
data.val_mask.sum().item()

500

In [17]:
# nodes against which to test
data.test_mask.sum().item()

1000

In [20]:
# let's do this
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x) # nonlinearity applied after calls (non-integrated)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [21]:
print(torch.cuda.is_available())

True


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [23]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [25]:
# eval on test nodes
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'accuracy: {acc:.4f}')

accuracy: 0.8090
